In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-270m")
model = AutoModelForCausalLM.from_pretrained("google/gemma-3-270m")

/Users/likhit/Documents/Coding/NLP/NLP_basics/nlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(device)
model.to(device)
model

mps


Gemma3ForCausalLM(
  (model): Gemma3TextModel(
    (embed_tokens): Gemma3TextScaledWordEmbedding(262144, 640, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x Gemma3DecoderLayer(
        (self_attn): Gemma3Attention(
          (q_proj): Linear(in_features=640, out_features=1024, bias=False)
          (k_proj): Linear(in_features=640, out_features=256, bias=False)
          (v_proj): Linear(in_features=640, out_features=256, bias=False)
          (o_proj): Linear(in_features=1024, out_features=640, bias=False)
          (q_norm): Gemma3RMSNorm((256,), eps=1e-06)
          (k_norm): Gemma3RMSNorm((256,), eps=1e-06)
        )
        (mlp): Gemma3MLP(
          (gate_proj): Linear(in_features=640, out_features=2048, bias=False)
          (up_proj): Linear(in_features=640, out_features=2048, bias=False)
          (down_proj): Linear(in_features=2048, out_features=640, bias=False)
          (act_fn): GELUTanh()
        )
        (input_layernorm): Gemma3RMSNorm((640,), eps=1e-06)

In [3]:
prompt = "What is Machine Learning is the best that humans have ever bulit  in there lives and tell us now check out of this really work can you  help me to start ?"
inputs = tokenizer(prompt , return_tensors="pt").to(device)
print(inputs["input_ids"].shape)
print(inputs["attention_mask"].shape)

output = model(**inputs, use_cache=True)

torch.Size([1, 36])
torch.Size([1, 36])


In [4]:
# checking the next token
print(output.logits.shape)
last_token_logits = output.logits[:,-1,:]
print(last_token_logits.shape)
next_token_id = torch.softmax(last_token_logits,dim=-1)
print(next_token_id.shape)

torch.Size([1, 36, 262144])
torch.Size([1, 262144])
torch.Size([1, 262144])


In [11]:
# KV key_values stored in it.
past_key_values = output.past_key_values
print((past_key_values))
k , v = output.past_key_values[0]
print(k.shape)
print(v.shape)
print(k.dtype,v.dtype)

size = 1 * 1 * 36 * 256 * 4
print(size/1000, "KB")
# Batch_size, n_head , seq_len , dim

DynamicCache(layers=[DynamicSlidingWindowLayer, DynamicSlidingWindowLayer, DynamicSlidingWindowLayer, DynamicSlidingWindowLayer, DynamicSlidingWindowLayer, DynamicLayer, DynamicSlidingWindowLayer, DynamicSlidingWindowLayer, DynamicSlidingWindowLayer, DynamicSlidingWindowLayer, DynamicSlidingWindowLayer, DynamicLayer, DynamicSlidingWindowLayer, DynamicSlidingWindowLayer, DynamicSlidingWindowLayer, DynamicSlidingWindowLayer, DynamicSlidingWindowLayer, DynamicLayer])
torch.Size([1, 1, 36, 256])
torch.Size([1, 1, 36, 256])
torch.float32 torch.float32
36.864 KB


In [ ]:
#  next word predictions.
logits_val , logis_idx = torch.topk(next_token_id,10,dim=-1)

print(logits_val.shape)
print(logis_idx)
top_k_probs = logits_val / torch.sum(logits_val,dim=-1)
print(top_k_probs.shape)
probs = torch.multinomial(top_k_probs,1)
print(probs.shape)
print(probs)
next_token_id = torch.gather(logis_idx, dim=-1, index=probs)
print(next_token_id)

# last token is 
word = tokenizer.decode(next_token_id[0])
print(word)

In [ ]:
# he new k v values get attached to this KV cahce ) past_key_values and we get the logits of the output_next .. 
output_next = model(input_ids=next_token_id,past_key_values=past_key_values)

In [112]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device = "cpu"   # or "cuda"
model = AutoModelForCausalLM.from_pretrained("google/gemma-3-270m").to(device)
tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-270m")

prompts = [
    "What is machine Learning?",
    "Tell me a joke about cats.",
    "Who are you?"
]

sep = tokenizer.eos_token  # SAFE separator
print("SEP TOKEN:", sep)

# 1) Join prompts
joined = sep.join(prompts)
print("\nJoined prompt:")
print(joined)

tokens = tokenizer(joined, return_tensors="pt",add_special_tokens=False).to(device)
print( "Length of input ids: ", len(tokens["input_ids"][0]))

# 2. Track each prompt's token range

prompt_ranges = []
cur = 0

sep_id = tokenizer(sep, add_special_tokens=False)
print("Separator ID:", sep_id["input_ids"])

sep_len = sep_id["input_ids"][0]
print("Seq Len of sep:", sep_len)

for p in prompts:
    tok_len = len(tokenizer(p,add_special_tokens=False)["input_ids"])
    prompt_ranges.append((cur, cur + tok_len))
    cur += tok_len + sep_len

print("Token ranges:", prompt_ranges)

# 3. Build block diagonal attention mask
seq_len = tokens["input_ids"].shape[1]

print("Seq Len:", seq_len)

attn_mask = torch.zeros((seq_len, seq_len), dtype=torch.float, device=device)

for (start, end) in prompt_ranges:
    attn_mask[start:end, start:end] = 1

# 4. Expand to correct shape: [batch, heads, seq_len, seq_len]
num_heads = model.config.num_attention_heads
attn_mask = attn_mask.unsqueeze(0).unsqueeze(0)   # [1,1,seq,seq]
attn_mask = attn_mask.repeat(1, num_heads, 1, 1)  # [1,h,seq,seq]

print("Final attn_mask shape:", attn_mask.shape)

# 5. Forward pass
with torch.no_grad():
    outputs = model(
        input_ids=tokens["input_ids"],
        attention_mask=attn_mask,
        use_cache=True,
    )

print("Success — prompts packed WITHOUT cross-attending!")

SEP TOKEN: <eos>

Joined prompt:
What is machine Learning?<eos>Tell me a joke about cats.<eos>Who are you?
Length of input ids:  18
Separator ID: [1]
Seq Len of sep: 1
Token ranges: [(0, 5), (6, 13), (14, 18)]
Seq Len: 18
Final attn_mask shape: torch.Size([1, 4, 18, 18])
Success — prompts packed WITHOUT cross-attending!


In [113]:
# Extract KV cache for first prompt
past_kv = outputs.past_key_values
first_prompt_len = len(tokenizer(prompts[0], add_special_tokens=False)["input_ids"])
print(f"Prompt1 token length = {first_prompt_len}")

# ✅ CORRECT: Don't stack - just slice each layer's tuple
past = []
for layer_kv in past_kv:
    k, v = layer_kv  # Each is [batch, heads, seq, head_dim]
    # Slice to keep only first prompt's tokens
    k_sliced = k[:, :, 0:first_prompt_len, :]
    v_sliced = v[:, :, 0:first_prompt_len, :]
    past.append((k_sliced, v_sliced))

print("Sample k shape:", past[0][0].shape)  # Should be [1, 4, 6, 256]
print("Sample v shape:", past[0][1].shape)

Prompt1 token length = 5
Sample k shape: torch.Size([1, 1, 5, 256])
Sample v shape: torch.Size([1, 1, 5, 256])


In [114]:
# Let's see what the prefix actually is
prefix_text = tokenizer.decode(tokens["input_ids"][0, :first_prompt_len])
print("Prefix text:", repr(prefix_text))
print("Prefix tokens:", tokens["input_ids"][0, :first_prompt_len].tolist())

# Let's try without cache to see what SHOULD be generated
with torch.no_grad():
    out_no_cache = model.generate(
        tokens["input_ids"][:, :first_prompt_len],
        max_new_tokens=30,
        do_sample=False,
    )
print("Without cache:", tokenizer.decode(out_no_cache[0], skip_special_tokens=True))

Prefix text: 'What is machine Learning?'
Prefix tokens: [3689, 563, 5464, 19180, 236881]
Without cache: What is machine Learning???????????????????????????????


In [115]:
# Let's verify the cache was extracted correctly
print("Original output cache info:")
print("  Number of layers:", len(outputs.past_key_values))
print("  Layer 0 K shape:", outputs.past_key_values[0][0].shape)
print("  Layer 0 V shape:", outputs.past_key_values[0][1].shape)

print("\nExtracted cache info:")
print("  Number of layers:", len(past))
print("  Layer 0 K shape:", past[0][0].shape)
print("  Layer 0 V shape:", past[0][1].shape)

# Verify the extraction is correct by comparing
print("\nAre they the same?")
print("  K match:", torch.allclose(outputs.past_key_values[0][0][:, :, :first_prompt_len, :], past[0][0]))
print("  V match:", torch.allclose(outputs.past_key_values[0][1][:, :, :first_prompt_len, :], past[0][1]))

# Now let's try a simpler approach: just continue from the original cache

Original output cache info:
  Number of layers: 18
  Layer 0 K shape: torch.Size([1, 1, 18, 256])
  Layer 0 V shape: torch.Size([1, 1, 18, 256])

Extracted cache info:
  Number of layers: 18
  Layer 0 K shape: torch.Size([1, 1, 5, 256])
  Layer 0 V shape: torch.Size([1, 1, 5, 256])

Are they the same?
  K match: True
  V match: True


In [110]:
from transformers.cache_utils import DynamicCache

def decode_with_kv_final(tokenizer, model, device, original_cache, prefix_len, max_new=30, temperature=1.0, top_k=50):
    generated = []
    
    # Extract the FULL prefix cache (all 6 tokens)
    cache = DynamicCache()
    for layer_idx, (k, v) in enumerate(original_cache):
        k_sliced = k[:, :, :prefix_len, :]
        v_sliced = v[:, :, :prefix_len, :]
        cache.update(k_sliced, v_sliced, layer_idx)
    
    print(f"Starting generation:")
    print(f"  Cache length: {cache.get_seq_length()}")
    
    # To generate the NEXT token after the cached prefix, we need to:
    # Option 1: Pass an empty/dummy token (doesn't work well)
    # Option 2: Do one forward pass with the last cached token to get logits
    
    # Get logits for next token by looking at what was computed in original forward pass
    # The logits at position [prefix_len-1] tell us what comes after the prefix
    with torch.no_grad():
        # Re-run just the last token with the cache of previous tokens
        input_ids = tokens["input_ids"][:, prefix_len-1:prefix_len].to(device)
        
        # Use cache of first prefix_len-1 tokens
        cache_partial = DynamicCache()
        for layer_idx, (k, v) in enumerate(original_cache):
            k_sliced = k[:, :, :prefix_len-1, :]
            v_sliced = v[:, :, :prefix_len-1, :]
            cache_partial.update(k_sliced, v_sliced, layer_idx)
        
        out = model(
            input_ids=input_ids,
            past_key_values=cache_partial,
            use_cache=True,
        )
    
    # Sample first token with temperature
    logits = out.logits[:, -1, :] / temperature
    
    # Top-k sampling
    if top_k > 0:
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = float('-inf')
    
    probs = torch.softmax(logits, dim=-1)
    next_token = torch.multinomial(probs, num_samples=1)
    
    generated.append(next_token.item())
    print(f"First token: {next_token.item()} -> '{tokenizer.decode(next_token[0])}'")
    
    if next_token.item() == tokenizer.eos_token_id:
        return tokenizer.decode(generated, skip_special_tokens=True)
    
    # Continue generation with updated cache
    cache = out.past_key_values
    input_ids = next_token
    
    for step in range(1, max_new):
        with torch.no_grad():
            out = model(
                input_ids=input_ids,
                past_key_values=cache,
                use_cache=True,
            )

        # Sample with temperature
        logits = out.logits[:, -1, :] / temperature
        
        if top_k > 0:
            indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
            logits[indices_to_remove] = float('-inf')
        
        probs = torch.softmax(logits, dim=-1)
        next_token = torch.multinomial(probs, num_samples=1)
        
        generated.append(next_token.item())
        
        if step < 5:
            print(f"Step {step}: token = {next_token.item()} -> '{tokenizer.decode(next_token[0])}'")
        
        if next_token.item() == tokenizer.eos_token_id:
            break
            
        input_ids = next_token
        cache = out.past_key_values

    return tokenizer.decode(generated, skip_special_tokens=True)

result = decode_with_kv_final(
    tokenizer=tokenizer,
    model=model,
    device="cpu",
    original_cache=outputs.past_key_values,
    prefix_len=first_prompt_len,
    max_new=30,
    temperature=0.8,
    top_k=50
)
print("\nFinal generated:", result)

Starting generation:
  Cache length: 5
First token: 1 -> '<eos>'

Final generated: 


In [116]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.cache_utils import DynamicCache

device = "cpu"
model = AutoModelForCausalLM.from_pretrained("google/gemma-3-270m").to(device)
tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-270m")

prompts = [
    "What is machine learning?",
    "Tell me a joke about cats.",
    "Who are you?"
]

print("=" * 60)
print("STEP 1: Process each prompt SEPARATELY (no packing)")
print("=" * 60)

# Process each prompt individually to get clean KV caches
all_caches = []
all_prompt_lens = []

for i, prompt in enumerate(prompts):
    print(f"\nProcessing prompt {i+1}: '{prompt}'")
    
    # Tokenize this prompt alone
    tokens_single = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).to(device)
    prompt_len = tokens_single["input_ids"].shape[1]
    all_prompt_lens.append(prompt_len)
    
    print(f"  Token length: {prompt_len}")
    print(f"  Tokens: {tokens_single['input_ids'][0].tolist()}")
    
    # Forward pass with NORMAL attention (no blocking)
    with torch.no_grad():
        outputs = model(
            input_ids=tokens_single["input_ids"],
            use_cache=True,
        )
    
    # Extract and store the KV cache for this prompt
    cache_for_prompt = []
    for layer_kv in outputs.past_key_values:
        k, v = layer_kv
        cache_for_prompt.append((k, v))
    
    all_caches.append(cache_for_prompt)
    
    print(f"  Cache shape: K={cache_for_prompt[0][0].shape}, V={cache_for_prompt[0][1].shape}")

print("\n" + "=" * 60)
print("STEP 2: Generate from each cached prompt")
print("=" * 60)

def decode_with_clean_cache(prompt_text, cache_list, prompt_len, tokenizer, model, device, max_new=30, temperature=0.8, top_k=50):
    """Generate using a clean KV cache from individual prompt processing"""
    
    generated = []
    
    # Convert to DynamicCache
    cache_partial = DynamicCache()
    for layer_idx, (k, v) in enumerate(cache_list):
        # Use cache of first prompt_len-1 tokens
        k_sliced = k[:, :, :prompt_len-1, :]
        v_sliced = v[:, :, :prompt_len-1, :]
        cache_partial.update(k_sliced, v_sliced, layer_idx)
    
    # Get the last token to process with the cache
    tokens_single = tokenizer(prompt_text, return_tensors="pt", add_special_tokens=False).to(device)
    input_ids = tokens_single["input_ids"][:, prompt_len-1:prompt_len]
    
    print(f"\n  Prompt: '{prompt_text}'")
    print(f"  Cache length: {cache_partial.get_seq_length()}")
    print(f"  Last token: '{tokenizer.decode(input_ids[0])}'")
    
    # First token generation
    with torch.no_grad():
        out = model(
            input_ids=input_ids,
            past_key_values=cache_partial,
            use_cache=True,
        )
    
    # Sample first token
    logits = out.logits[:, -1, :] / temperature
    if top_k > 0:
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = float('-inf')
    
    probs = torch.softmax(logits, dim=-1)
    next_token = torch.multinomial(probs, num_samples=1)
    generated.append(next_token.item())
    
    print(f"  First generated: '{tokenizer.decode(next_token[0])}'")
    
    if next_token.item() == tokenizer.eos_token_id:
        return tokenizer.decode(generated, skip_special_tokens=True)
    
    # Continue generation
    cache = out.past_key_values
    input_ids = next_token
    
    for step in range(1, max_new):
        with torch.no_grad():
            out = model(
                input_ids=input_ids,
                past_key_values=cache,
                use_cache=True,
            )
        
        logits = out.logits[:, -1, :] / temperature
        if top_k > 0:
            indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
            logits[indices_to_remove] = float('-inf')
        
        probs = torch.softmax(logits, dim=-1)
        next_token = torch.multinomial(probs, num_samples=1)
        generated.append(next_token.item())
        
        if next_token.item() == tokenizer.eos_token_id:
            break
        
        input_ids = next_token
        cache = out.past_key_values
    
    return tokenizer.decode(generated, skip_special_tokens=True)

# Generate from each prompt's cache
for i, (prompt, cache, prompt_len) in enumerate(zip(prompts, all_caches, all_prompt_lens)):
    result = decode_with_clean_cache(
        prompt_text=prompt,
        cache_list=cache,
        prompt_len=prompt_len,
        tokenizer=tokenizer,
        model=model,
        device=device,
        max_new=30,
        temperature=0.8,
        top_k=50
    )
    print(f"  Generated: {result}")
    print()

STEP 1: Process each prompt SEPARATELY (no packing)

Processing prompt 1: 'What is machine learning?'
  Token length: 5
  Tokens: [3689, 563, 5464, 4735, 236881]
  Cache shape: K=torch.Size([1, 1, 5, 256]), V=torch.Size([1, 1, 5, 256])

Processing prompt 2: 'Tell me a joke about cats.'
  Token length: 7
  Tokens: [54593, 786, 496, 31481, 1003, 22797, 236761]
  Cache shape: K=torch.Size([1, 1, 7, 256]), V=torch.Size([1, 1, 7, 256])

Processing prompt 3: 'Who are you?'
  Token length: 4
  Tokens: [15938, 659, 611, 236881]
  Cache shape: K=torch.Size([1, 1, 4, 256]), V=torch.Size([1, 1, 4, 256])

STEP 2: Generate from each cached prompt

  Prompt: 'What is machine learning?'
  Cache length: 4
  Last token: '?'
  First generated: '?'
  Generated: ???????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????


  Prompt: 'Tell me a joke

In [117]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.cache_utils import DynamicCache

device = "cpu"
model = AutoModelForCausalLM.from_pretrained("google/gemma-3-270m").to(device)
tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-270m")

print("=" * 60)
print("DEBUG: Compare normal generation vs KV cache generation")
print("=" * 60)

prompt = "What is machine learning?"
print(f"\nPrompt: '{prompt}'")

# Tokenize
tokens = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).to(device)
prompt_len = tokens["input_ids"].shape[1]
print(f"Token length: {prompt_len}")
print(f"Tokens: {tokens['input_ids'][0].tolist()}")

# ============================================================
# Test 1: Normal generation (baseline)
# ============================================================
print("\n" + "=" * 60)
print("Test 1: NORMAL generation (no cache manipulation)")
print("=" * 60)

with torch.no_grad():
    output_normal = model.generate(
        tokens["input_ids"],
        max_new_tokens=20,
        do_sample=False,  # Greedy for reproducibility
        pad_token_id=tokenizer.eos_token_id,
    )

generated_normal = tokenizer.decode(output_normal[0][prompt_len:], skip_special_tokens=True)
print(f"Generated: {generated_normal}")

# ============================================================
# Test 2: Get KV cache and check logits
# ============================================================
print("\n" + "=" * 60)
print("Test 2: Forward pass and examine outputs")
print("=" * 60)

with torch.no_grad():
    outputs = model(
        input_ids=tokens["input_ids"],
        use_cache=True,
    )

# Check what the next token SHOULD be according to the logits
next_token_id = torch.argmax(outputs.logits[0, -1, :]).item()
print(f"Next token from logits: {next_token_id} -> '{tokenizer.decode([next_token_id])}'")

# ============================================================
# Test 3: Try to reproduce using manual KV cache
# ============================================================
print("\n" + "=" * 60)
print("Test 3: Manual generation with KV cache")
print("=" * 60)

# Extract cache
cache = DynamicCache()
for layer_idx, (k, v) in enumerate(outputs.past_key_values):
    cache.update(k, v, layer_idx)

print(f"Cache length: {cache.get_seq_length()}")

# Now try to generate next token using the cache
# The cache already contains ALL tokens, so we just need to sample from logits
generated_ids = []

# First token: use the logits we already have
next_token = torch.argmax(outputs.logits[0, -1, :], keepdim=True).unsqueeze(0)
generated_ids.append(next_token.item())
print(f"Step 0: {next_token.item()} -> '{tokenizer.decode(next_token[0])}'")

# Continue generation
input_ids = next_token
for step in range(1, 20):
    with torch.no_grad():
        out = model(
            input_ids=input_ids,
            past_key_values=cache,
            use_cache=True,
        )
    
    next_token = torch.argmax(out.logits[0, -1, :], keepdim=True).unsqueeze(0)
    generated_ids.append(next_token.item())
    
    if step < 5:
        print(f"Step {step}: {next_token.item()} -> '{tokenizer.decode(next_token[0])}'")
    
    if next_token.item() == tokenizer.eos_token_id:
        break
    
    input_ids = next_token
    cache = out.past_key_values

generated_manual = tokenizer.decode(generated_ids, skip_special_tokens=True)
print(f"\nManual generation: {generated_manual}")

# ============================================================
# Test 4: What you were trying to do (partial cache)
# ============================================================
print("\n" + "=" * 60)
print("Test 4: Your approach (partial cache + last token)")
print("=" * 60)

# Create partial cache (first n-1 tokens)
cache_partial = DynamicCache()
for layer_idx, (k, v) in enumerate(outputs.past_key_values):
    k_sliced = k[:, :, :prompt_len-1, :]
    v_sliced = v[:, :, :prompt_len-1, :]
    cache_partial.update(k_sliced, v_sliced, layer_idx)

print(f"Partial cache length: {cache_partial.get_seq_length()}")

# Process last token with partial cache
last_token = tokens["input_ids"][:, -1:].to(device)
print(f"Last token: {last_token.item()} -> '{tokenizer.decode(last_token[0])}'")

with torch.no_grad():
    out = model(
        input_ids=last_token,
        past_key_values=cache_partial,
        use_cache=True,
    )

next_token_id = torch.argmax(out.logits[0, -1, :]).item()
print(f"Next token: {next_token_id} -> '{tokenizer.decode([next_token_id])}'")

# Continue generation
generated_ids_partial = [next_token_id]
input_ids = torch.tensor([[next_token_id]], device=device)
cache_new = out.past_key_values

for step in range(1, 20):
    with torch.no_grad():
        out = model(
            input_ids=input_ids,
            past_key_values=cache_new,
            use_cache=True,
        )
    
    next_token = torch.argmax(out.logits[0, -1, :]).item()
    generated_ids_partial.append(next_token)
    
    if step < 5:
        print(f"Step {step}: {next_token} -> '{tokenizer.decode([next_token])}'")
    
    if next_token == tokenizer.eos_token_id:
        break
    
    input_ids = torch.tensor([[next_token]], device=device)
    cache_new = out.past_key_values

generated_partial = tokenizer.decode(generated_ids_partial, skip_special_tokens=True)
print(f"\nPartial cache generation: {generated_partial}")

# ============================================================
# Summary
# ============================================================
print("\n" + "=" * 60)
print("SUMMARY")
print("=" * 60)
print(f"Normal generation:       {generated_normal}")
print(f"Manual with full cache:  {generated_manual}")
print(f"Your approach (partial): {generated_partial}")

DEBUG: Compare normal generation vs KV cache generation

Prompt: 'What is machine learning?'
Token length: 5
Tokens: [3689, 563, 5464, 4735, 236881]

Test 1: NORMAL generation (no cache manipulation)
Generated: ????????????????????

Test 2: Forward pass and examine outputs
Next token from logits: 236881 -> '?'

Test 3: Manual generation with KV cache
Cache length: 5
Step 0: 236881 -> '?'
Step 1: 236881 -> '?'
Step 2: 236881 -> '?'
Step 3: 236881 -> '?'
Step 4: 236881 -> '?'

Manual generation: ????????????????????

Test 4: Your approach (partial cache + last token)
Partial cache length: 4
Last token: 236881 -> '?'
Next token: 236881 -> '?'
Step 1: 236881 -> '?'
Step 2: 236881 -> '?'
Step 3: 236881 -> '?'
Step 4: 236881 -> '?'

Partial cache generation: ????????????????????

SUMMARY
Normal generation:       ????????????????????
Manual with full cache:  ????????????????????
Your approach (partial): ????????????????????


In [119]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.cache_utils import DynamicCache

device = "cpu"
model = AutoModelForCausalLM.from_pretrained("google/gemma-3-270m").to(device)
tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-270m")

# ✅ Format prompts properly for Gemma
prompts = [
    "What is machine learning?",
    "Tell me a joke about cats.",
    "Who are you?"
]

sep = tokenizer.eos_token
print("SEP TOKEN:", sep)
print("BOS TOKEN ID:", tokenizer.bos_token_id)

# ============================================================
# CORRECT APPROACH: Add BOS token and process individually
# ============================================================
print("\n" + "=" * 60)
print("Processing prompts with proper tokenization")
print("=" * 60)

all_caches = []
all_prompt_lens = []
all_tokens = []

for i, prompt in enumerate(prompts):
    print(f"\nPrompt {i+1}: '{prompt}'")
    
    # ✅ Add special tokens (BOS)
    tokens_single = tokenizer(prompt, return_tensors="pt", add_special_tokens=True).to(device)
    prompt_len = tokens_single["input_ids"].shape[1]
    all_prompt_lens.append(prompt_len)
    all_tokens.append(tokens_single["input_ids"])
    
    print(f"  Tokens: {tokens_single['input_ids'][0].tolist()}")
    print(f"  Length: {prompt_len}")
    
    # Forward pass
    with torch.no_grad():
        outputs = model(
            input_ids=tokens_single["input_ids"],
            use_cache=True,
        )
    
    # Check what it wants to generate
    next_token_id = torch.argmax(outputs.logits[0, -1, :]).item()
    print(f"  Next token prediction: '{tokenizer.decode([next_token_id])}'")
    
    # Store cache
    cache_for_prompt = []
    for layer_kv in outputs.past_key_values:
        k, v = layer_kv
        cache_for_prompt.append((k, v))
    
    all_caches.append(cache_for_prompt)

# ============================================================
# Generate from each cached prompt
# ============================================================
print("\n" + "=" * 60)
print("Generating from cached prompts")
print("=" * 60)

def generate_from_cache(prompt_tokens, cache_list, prompt_len, tokenizer, model, device, max_new=30):
    """Generate using KV cache"""
    generated = []
    
    # Convert to DynamicCache (use full cache this time)
    cache = DynamicCache()
    for layer_idx, (k, v) in enumerate(cache_list):
        cache.update(k, v, layer_idx)
    
    print(f"  Cache length: {cache.get_seq_length()}")
    
    # Use the logits from the initial forward pass to get first token
    # We need to recompute to get the logits
    with torch.no_grad():
        out = model(
            input_ids=prompt_tokens,
            use_cache=True,
        )
    
    # Get first token
    next_token = torch.argmax(out.logits[0, -1, :], keepdim=True).unsqueeze(0)
    generated.append(next_token.item())
    
    print(f"  First token: '{tokenizer.decode(next_token[0])}'")
    
    if next_token.item() == tokenizer.eos_token_id:
        return tokenizer.decode(generated, skip_special_tokens=True)
    
    # Continue with cache
    input_ids = next_token
    cache = out.past_key_values
    
    for step in range(1, max_new):
        with torch.no_grad():
            out = model(
                input_ids=input_ids,
                past_key_values=cache,
                use_cache=True,
            )
        
        # Use sampling for better results
        logits = out.logits[0, -1, :] / 0.8
        probs = torch.softmax(logits, dim=-1)
        
        # Top-k sampling
        top_k = 50
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = float('-inf')
        probs = torch.softmax(logits, dim=-1)
        
        next_token = torch.multinomial(probs, num_samples=1)
        generated.append(next_token.item())
        
        if step < 3:
            print(f"  Step {step}: '{tokenizer.decode(next_token[0])}'")
        
        if next_token.item() == tokenizer.eos_token_id:
            break
        
        input_ids = next_token.unsqueeze(0)
        cache = out.past_key_values
    
    return tokenizer.decode(generated, skip_special_tokens=True)

# Generate from each
for i, (prompt, cache, prompt_len, tokens) in enumerate(zip(prompts, all_caches, all_prompt_lens, all_tokens)):
    print(f"\nPrompt {i+1}: '{prompt}'")
    result = generate_from_cache(
        prompt_tokens=tokens,
        cache_list=cache,
        prompt_len=prompt_len,
        tokenizer=tokenizer,
        model=model,
        device=device,
        max_new=30
    )
    print(f"  Generated: {result}\n")

SEP TOKEN: <eos>
BOS TOKEN ID: 2

Processing prompts with proper tokenization

Prompt 1: 'What is machine learning?'
  Tokens: [2, 3689, 563, 5464, 4735, 236881]
  Length: 6
  Next token prediction: '

'

Prompt 2: 'Tell me a joke about cats.'
  Tokens: [2, 54593, 786, 496, 31481, 1003, 22797, 236761]
  Length: 8
  Next token prediction: '

'

Prompt 3: 'Who are you?'
  Tokens: [2, 15938, 659, 611, 236881]
  Length: 5
  Next token prediction: '

'

Generating from cached prompts

Prompt 1: 'What is machine learning?'
  Cache length: 6
  First token: '

'
  Step 1: 'Machine'
  Step 2: ' learning'
  Generated: 

Machine learning is a technique that has helped us develop a lot of important things in technology. But, the term, machine learning, is not actually


Prompt 2: 'Tell me a joke about cats.'
  Cache length: 8
  First token: '

'
  Step 1: 'If'
  Step 2: ' I'
  Generated: 

If I did, the joke would be:

“cats don't know what they're talking about. They just know their owners don




In [127]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.cache_utils import DynamicCache

device = "cpu"
model = AutoModelForCausalLM.from_pretrained("google/gemma-3-270m").to(device)
tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-270m")

prompts = [
    "What is machine learning?",
    "Can you tell me a joke about cats.",
    "Do you know anything about India?"
]

print("=" * 60)
print("PROMPT PACKING WITH KV CACHE EXTRACTION")
print("=" * 60)

# ============================================================
# Step 1: Tokenize each prompt WITH BOS token
# ============================================================
print("\nStep 1: Tokenize prompts individually (to get lengths)")

prompt_tokens = []
prompt_lengths = []

for i, prompt in enumerate(prompts):
    tokens = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
    prompt_tokens.append(tokens["input_ids"][0])
    prompt_lengths.append(len(tokens["input_ids"][0]))
    print(f"  Prompt {i+1}: '{prompt}' -> {prompt_lengths[i]} tokens")

# ============================================================
# Step 2: Pack prompts (concatenate with proper separation)
# ============================================================
print("\nStep 2: Pack prompts into single sequence")

# Concatenate all prompts (they already have BOS, no need for separator between)
# OR use EOS as separator if you want explicit boundaries
sep_token_id = tokenizer.eos_token_id

packed_tokens = []
prompt_ranges = []
current_pos = 0

for i, tokens in enumerate(prompt_tokens):
    start = current_pos
    end = current_pos + len(tokens)
    prompt_ranges.append((start, end))
    
    packed_tokens.extend(tokens.tolist())
    
    # Add separator after each prompt (except last)
    if i < len(prompt_tokens) - 1:
        packed_tokens.append(sep_token_id)
        current_pos = end + 1  # +1 for separator
    else:
        current_pos = end

# Convert to tensor
packed_input_ids = torch.tensor([packed_tokens], device=device)
seq_len = packed_input_ids.shape[1]

print(f"  Packed sequence length: {seq_len}")
print(f"  Prompt ranges: {prompt_ranges}")

# ============================================================
# Step 3: Build block-diagonal attention mask
# ============================================================
print("\nStep 3: Build block-diagonal attention mask")

attn_mask = torch.zeros((seq_len, seq_len), dtype=torch.float, device=device)

for start, end in prompt_ranges:
    attn_mask[start:end, start:end] = 1.0

# Expand for multi-head attention
num_heads = model.config.num_attention_heads
attn_mask = attn_mask.unsqueeze(0).unsqueeze(0)  # [1, 1, seq, seq]
attn_mask = attn_mask.repeat(1, num_heads, 1, 1)  # [1, heads, seq, seq]

print(f"  Attention mask shape: {attn_mask.shape}")

# ============================================================
# Step 4: Single forward pass with packed prompts
# ============================================================
print("\nStep 4: Forward pass with prompt packing")

with torch.no_grad():
    outputs = model(
        input_ids=packed_input_ids,
        attention_mask=attn_mask,
        use_cache=True,
    )

print("  ✓ Forward pass complete")

# ============================================================
# Step 5: Extract individual KV caches for each prompt
# ============================================================
print("\nStep 5: Extract KV caches for each prompt")

all_caches = []

for i, (start, end) in enumerate(prompt_ranges):
    cache_for_prompt = []
    
    for layer_kv in outputs.past_key_values:
        k, v = layer_kv  # Shape: [batch, heads, seq, head_dim]
        
        # Extract only this prompt's tokens
        k_sliced = k[:, :, start:end, :]
        v_sliced = v[:, :, start:end, :]
        
        cache_for_prompt.append((k_sliced, v_sliced))
    
    all_caches.append(cache_for_prompt)
    print(f"  Prompt {i+1}: Extracted cache with {end-start} tokens")

# ============================================================
# Step 6: Generate from each extracted cache
# ============================================================
print("\n" + "=" * 60)
print("Step 6: Generate from extracted caches")
print("=" * 60)

def generate_from_extracted_cache(prompt_text, cache_list, tokenizer, model, device, max_new=30):
    """Generate using extracted KV cache from packed batch"""
    
    generated = []
    
    # Tokenize prompt to know the token count
    tokens = tokenizer(prompt_text, return_tensors="pt", add_special_tokens=True).to(device)
    prompt_len = tokens["input_ids"].shape[1]
    
    print(f"\n  Prompt: '{prompt_text}'")
    print(f"  Prompt length: {prompt_len} tokens")
    
    # ✅ CORRECT: Use cache of first (n-1) tokens, process last token
    cache_partial = DynamicCache()
    for layer_idx, (k, v) in enumerate(cache_list):
        # Use all but the last token
        k_partial = k[:, :, :, :]
        v_partial = v[:, :, :, :]
        cache_partial.update(k_partial, v_partial, layer_idx)
    
    print(f"  Using cache length: {cache_partial.get_seq_length()}")
    
    # Process only the LAST token with the cached context
    last_token = tokens["input_ids"][:, -1:]
    print(f"  Processing last token: '{tokenizer.decode(last_token[0])}'")
    
    with torch.no_grad():
        out = model(
            input_ids=last_token,
            past_key_values=cache_partial,
            use_cache=True,
        )
    
    # Sample first NEW token
    logits = out.logits[0, -1, :] / 0.8
    probs = torch.softmax(logits, dim=-1)
    
    # Top-k sampling
    top_k = 50
    indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
    logits[indices_to_remove] = float('-inf')
    probs = torch.softmax(logits, dim=-1)
    
    next_token = torch.multinomial(probs, num_samples=1)
    generated.append(next_token.item())
    
    print(f"  First generated token: '{tokenizer.decode(next_token[0])}'")
    
    if next_token.item() == tokenizer.eos_token_id:
        return tokenizer.decode(generated, skip_special_tokens=True)
    
    # Continue generation with the updated cache
    cache = out.past_key_values
    input_ids = next_token.unsqueeze(0)
    
    for step in range(1, max_new):
        with torch.no_grad():
            out = model(
                input_ids=input_ids,
                past_key_values=cache,
                use_cache=True,
            )
        
        logits = out.logits[0, -1, :] / 0.8
        probs = torch.softmax(logits, dim=-1)
        
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = float('-inf')
        probs = torch.softmax(logits, dim=-1)
        
        next_token = torch.multinomial(probs, num_samples=1)
        generated.append(next_token.item())
        
        if step < 3:
            print(f"  Step {step}: '{tokenizer.decode(next_token[0])}'")
        
        if next_token.item() == tokenizer.eos_token_id:
            break
        
        input_ids = next_token.unsqueeze(0)
        cache = out.past_key_values
    
    return tokenizer.decode(generated, skip_special_tokens=True)

# Generate from each extracted cache
for i, (prompt, cache) in enumerate(zip(prompts, all_caches)):
    result = generate_from_extracted_cache(
        prompt_text=prompt,
        cache_list=cache,
        tokenizer=tokenizer,
        model=model,
        device=device,
        max_new=50
    )
    print(f"  Generated: {result}\n")

PROMPT PACKING WITH KV CACHE EXTRACTION

Step 1: Tokenize prompts individually (to get lengths)
  Prompt 1: 'What is machine learning?' -> 6 tokens
  Prompt 2: 'Can you tell me a joke about cats.' -> 10 tokens
  Prompt 3: 'Do you know anything about India?' -> 8 tokens

Step 2: Pack prompts into single sequence
  Packed sequence length: 26
  Prompt ranges: [(0, 6), (7, 17), (18, 26)]

Step 3: Build block-diagonal attention mask
  Attention mask shape: torch.Size([1, 4, 26, 26])

Step 4: Forward pass with prompt packing
  ✓ Forward pass complete

Step 5: Extract KV caches for each prompt
  Prompt 1: Extracted cache with 6 tokens
  Prompt 2: Extracted cache with 10 tokens
  Prompt 3: Extracted cache with 8 tokens

Step 6: Generate from extracted caches

  Prompt: 'What is machine learning?'
  Prompt length: 6 tokens
  Using cache length: 6
  Processing last token: '?'
  First generated token: '

'
  Step 1: 'A'
  Step 2: ' method'
  Generated: 

A method is a series of steps to improve a

In [131]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.cache_utils import DynamicCache

device = "cpu"
model = AutoModelForCausalLM.from_pretrained("google/gemma-3-270m").to(device)
tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-270m")

prompts = [
    "What is machine learning?",
    "Can you tell me a joke about cats.",
    "Do you know anything about India?"
]

print("=" * 60)
print("PROMPT PACKING WITH KV CACHE EXTRACTION")
print("=" * 60)

# ============================================================
# Step 1: Tokenize each prompt WITH BOS token
# ============================================================
print("\nStep 1: Tokenize prompts individually (to get lengths)")

prompt_tokens = []
prompt_lengths = []

for i, prompt in enumerate(prompts):
    tokens = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
    prompt_tokens.append(tokens["input_ids"][0])
    prompt_lengths.append(len(tokens["input_ids"][0]))
    print(f"  Prompt {i+1}: '{prompt}' -> {prompt_lengths[i]} tokens")

# ============================================================
# Step 2: Pack prompts (concatenate with proper separation)
# ============================================================
print("\nStep 2: Pack prompts into single sequence")

# Concatenate all prompts (they already have BOS, no need for separator between)
# OR use EOS as separator if you want explicit boundaries
sep_token_id = tokenizer.eos_token_id

packed_tokens = []
prompt_ranges = []
current_pos = 0

for i, tokens in enumerate(prompt_tokens):
    start = current_pos
    end = current_pos + len(tokens)
    prompt_ranges.append((start, end))
    
    packed_tokens.extend(tokens.tolist())
    
    # Add separator after each prompt (except last)
    if i < len(prompt_tokens) - 1:
        packed_tokens.append(sep_token_id)
        current_pos = end + 1  # +1 for separator
    else:
        current_pos = end

# Convert to tensor
packed_input_ids = torch.tensor([packed_tokens], device=device)
seq_len = packed_input_ids.shape[1]

print(f"  Packed sequence length: {seq_len}")
print(f"  Prompt ranges: {prompt_ranges}")

# ============================================================
# Step 3: Build block-diagonal attention mask
# ============================================================
print("\nStep 3: Build block-diagonal attention mask")

attn_mask = torch.zeros((seq_len, seq_len), dtype=torch.float, device=device)

for start, end in prompt_ranges:
    attn_mask[start:end, start:end] = 1.0

# Expand for multi-head attention
num_heads = model.config.num_attention_heads
attn_mask = attn_mask.unsqueeze(0).unsqueeze(0)  # [1, 1, seq, seq]
attn_mask = attn_mask.repeat(1, num_heads, 1, 1)  # [1, heads, seq, seq]

print(f"  Attention mask shape: {attn_mask.shape}")

# ============================================================
# Step 4: Single forward pass with packed prompts
# ============================================================
print("\nStep 4: Forward pass with prompt packing")

with torch.no_grad():
    outputs = model(
        input_ids=packed_input_ids,
        attention_mask=attn_mask,
        use_cache=True,
    )

print("  ✓ Forward pass complete")

# ============================================================
# Step 5: Extract individual KV caches for each prompt
# ============================================================
print("\nStep 5: Extract KV caches for each prompt")

all_caches = []

for i, (start, end) in enumerate(prompt_ranges):
    cache_for_prompt = []
    
    for layer_kv in outputs.past_key_values:
        k, v = layer_kv  # Shape: [batch, heads, seq, head_dim]
        
        # Extract only this prompt's tokens
        k_sliced = k[:, :, start:end, :]
        v_sliced = v[:, :, start:end, :]
        
        cache_for_prompt.append((k_sliced, v_sliced))
    
    all_caches.append(cache_for_prompt)
    print(f"  Prompt {i+1}: Extracted cache with {end-start} tokens")

# ============================================================
# Step 6: Generate from each extracted cache
# ============================================================
print("\n" + "=" * 60)
print("Step 6: Generate from extracted caches")
print("=" * 60)

def generate_from_extracted_cache(prompt_text, cache_list, tokenizer, model, device, max_new=30):
    """Generate using extracted KV cache from packed batch"""
    
    generated = []
    
    # Tokenize prompt to know the token count
    tokens = tokenizer(prompt_text, return_tensors="pt", add_special_tokens=True).to(device)
    prompt_len = tokens["input_ids"].shape[1]
    
    print(f"\n  Prompt: '{prompt_text}'")
    print(f"  Prompt length: {prompt_len} tokens")
    
    # Convert to DynamicCache
    cache_partial = DynamicCache()
    for layer_idx, (k, v) in enumerate(cache_list):
        cache_partial.update(k, v, layer_idx)
    
    cache_len = cache_partial.get_seq_length()
    print(f"  Using cache length: {cache_len}")
    
    # Process only the LAST token with the cached context
    last_token = tokens["input_ids"][:, -1:]
    print(f"  Processing last token: '{tokenizer.decode(last_token[0])}'")
    
    # ✅ EXPLICIT position_ids for the last token
    # Since cache has cache_len tokens (positions 0 to cache_len-1),
    # the last token should be at position cache_len-1
    position_ids = torch.tensor([[cache_len - 1]], device=device)
    print(f"  Position ID for last token: {position_ids.item()}")
    
    with torch.no_grad():
        out = model(
            input_ids=last_token,
            past_key_values=cache_partial,
            position_ids=position_ids,  # ✅ Explicit position
            use_cache=True,
        )
    
    # Sample first NEW token
    logits = out.logits[0, -1, :] / 0.8
    probs = torch.softmax(logits, dim=-1)
    
    # Top-k sampling
    top_k = 50
    indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
    logits[indices_to_remove] = float('-inf')
    probs = torch.softmax(logits, dim=-1)
    
    next_token = torch.multinomial(probs, num_samples=1)
    generated.append(next_token.item())
    
    print(f"  First generated token: '{tokenizer.decode(next_token[0])}'")
    
    if next_token.item() == tokenizer.eos_token_id:
        return tokenizer.decode(generated, skip_special_tokens=True)
    
    # Continue generation with the updated cache
    cache = out.past_key_values
    input_ids = next_token.unsqueeze(0)
    current_position = cache_len  # Next position after cache
    
    for step in range(1, max_new):
        # ✅ EXPLICIT position_ids for new tokens
        position_ids = torch.tensor([[current_position]], device=device)
        
        with torch.no_grad():
            out = model(
                input_ids=input_ids,
                past_key_values=cache,
                position_ids=position_ids,  # ✅ Explicit position
                use_cache=True,
            )
        
        logits = out.logits[0, -1, :] / 0.8
        probs = torch.softmax(logits, dim=-1)
        
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = float('-inf')
        probs = torch.softmax(logits, dim=-1)
        
        next_token = torch.multinomial(probs, num_samples=1)
        generated.append(next_token.item())
        
        if step < 3:
            print(f"  Step {step} (pos {current_position}): '{tokenizer.decode(next_token[0])}'")
        
        if next_token.item() == tokenizer.eos_token_id:
            break
        
        input_ids = next_token.unsqueeze(0)
        cache = out.past_key_values
        current_position += 1  # ✅ Increment position
    
    return tokenizer.decode(generated, skip_special_tokens=True)

# Generate from each extracted cache
for i, (prompt, cache) in enumerate(zip(prompts, all_caches)):
    result = generate_from_extracted_cache(
        prompt_text=prompt,
        cache_list=cache,
        tokenizer=tokenizer,
        model=model,
        device=device,
        max_new=50
    )
    print(f"  Generated: {result}\n")

PROMPT PACKING WITH KV CACHE EXTRACTION

Step 1: Tokenize prompts individually (to get lengths)
  Prompt 1: 'What is machine learning?' -> 6 tokens
  Prompt 2: 'Can you tell me a joke about cats.' -> 10 tokens
  Prompt 3: 'Do you know anything about India?' -> 8 tokens

Step 2: Pack prompts into single sequence
  Packed sequence length: 26
  Prompt ranges: [(0, 6), (7, 17), (18, 26)]

Step 3: Build block-diagonal attention mask
  Attention mask shape: torch.Size([1, 4, 26, 26])

Step 4: Forward pass with prompt packing
  ✓ Forward pass complete

Step 5: Extract KV caches for each prompt
  Prompt 1: Extracted cache with 6 tokens
  Prompt 2: Extracted cache with 10 tokens
  Prompt 3: Extracted cache with 8 tokens

Step 6: Generate from extracted caches

  Prompt: 'What is machine learning?'
  Prompt length: 6 tokens
  Using cache length: 6
  Processing last token: '?'
  Position ID for last token: 5
  First generated token: '

'
  Step 1 (pos 6): 'This'
  Step 2 (pos 7): ' is'
  Generate

In [129]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.cache_utils import DynamicCache

def pack_prompts(prompts, tokenizer, device):
    """
    Pack multiple prompts into a single sequence with separators.
    
    Returns:
        packed_input_ids: Tensor of packed token IDs
        prompt_ranges: List of (start, end) tuples for each prompt
    """
    prompt_tokens = []
    prompt_lengths = []
    
    # Tokenize each prompt
    for prompt in prompts:
        tokens = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
        prompt_tokens.append(tokens["input_ids"][0])
        prompt_lengths.append(len(tokens["input_ids"][0]))
    
    # Pack prompts with EOS separator
    sep_token_id = tokenizer.eos_token_id
    packed_tokens = []
    prompt_ranges = []
    current_pos = 0
    
    for i, tokens in enumerate(prompt_tokens):
        start = current_pos
        end = current_pos + len(tokens)
        prompt_ranges.append((start, end))
        
        packed_tokens.extend(tokens.tolist())
        
        # Add separator after each prompt (except last)
        if i < len(prompt_tokens) - 1:
            packed_tokens.append(sep_token_id)
            current_pos = end + 1
        else:
            current_pos = end
    
    packed_input_ids = torch.tensor([packed_tokens], device=device)
    
    return packed_input_ids, prompt_ranges, prompt_lengths


def create_block_diagonal_mask(seq_len, prompt_ranges, num_heads, device):
    """
    Create block-diagonal attention mask to prevent cross-attention between prompts.
    
    Args:
        seq_len: Total sequence length
        prompt_ranges: List of (start, end) tuples for each prompt
        num_heads: Number of attention heads
        device: Device to create tensor on
        
    Returns:
        attention_mask: Tensor of shape [1, num_heads, seq_len, seq_len]
    """
    attn_mask = torch.zeros((seq_len, seq_len), dtype=torch.float, device=device)
    
    # Fill diagonal blocks for each prompt
    for start, end in prompt_ranges:
        attn_mask[start:end, start:end] = 1.0
    
    # Expand for multi-head attention
    attn_mask = attn_mask.unsqueeze(0).unsqueeze(0)  # [1, 1, seq, seq]
    attn_mask = attn_mask.repeat(1, num_heads, 1, 1)  # [1, heads, seq, seq]
    
    return attn_mask


def extract_prompt_caches(past_key_values, prompt_ranges):
    """
    Extract individual KV caches for each prompt from packed forward pass.
    
    Args:
        past_key_values: KV cache from model forward pass
        prompt_ranges: List of (start, end) tuples for each prompt
        
    Returns:
        all_caches: List of cache_list for each prompt
    """
    all_caches = []
    
    for start, end in prompt_ranges:
        cache_for_prompt = []
        
        for layer_kv in past_key_values:
            k, v = layer_kv  # Shape: [batch, heads, seq, head_dim]
            
            # Extract only this prompt's tokens
            k_sliced = k[:, :, start:end, :]
            v_sliced = v[:, :, start:end, :]
            
            cache_for_prompt.append((k_sliced, v_sliced))
        
        all_caches.append(cache_for_prompt)
    
    return all_caches


def generate_from_cache(
    prompt_text,
    cache_list,
    tokenizer,
    model,
    device,
    max_new=50,
    temperature=0.8,
    top_k=50
):
    """
    Generate text using extracted KV cache from packed batch.
    
    Args:
        prompt_text: Original prompt text
        cache_list: List of (K, V) tuples for each layer
        tokenizer: Tokenizer instance
        model: Model instance
        device: Device
        max_new: Maximum new tokens to generate
        temperature: Sampling temperature
        top_k: Top-k sampling parameter
        
    Returns:
        generated_text: Generated text string
    """
    generated = []
    
    # Tokenize prompt to know token count
    tokens = tokenizer(prompt_text, return_tensors="pt", add_special_tokens=True).to(device)
    prompt_len = tokens["input_ids"].shape[1]
    
    # Convert to DynamicCache
    cache = DynamicCache()
    for layer_idx, (k, v) in enumerate(cache_list):
        cache.update(k, v, layer_idx)
    
    cache_len = cache.get_seq_length()
    
    # Process last token with cached context
    last_token = tokens["input_ids"][:, -1:]
    position_ids = torch.tensor([[cache_len - 1]], device=device)
    
    with torch.no_grad():
        out = model(
            input_ids=last_token,
            past_key_values=cache,
            position_ids=position_ids,
            use_cache=True,
        )
    
    # Sample first token
    logits = out.logits[0, -1, :] / temperature
    probs = torch.softmax(logits, dim=-1)
    
    if top_k > 0:
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = float('-inf')
        probs = torch.softmax(logits, dim=-1)
    
    next_token = torch.multinomial(probs, num_samples=1)
    generated.append(next_token.item())
    
    if next_token.item() == tokenizer.eos_token_id:
        return tokenizer.decode(generated, skip_special_tokens=True)
    
    # Continue generation
    cache = out.past_key_values
    input_ids = next_token.unsqueeze(0)
    current_position = cache_len
    
    for _ in range(1, max_new):
        position_ids = torch.tensor([[current_position]], device=device)
        
        with torch.no_grad():
            out = model(
                input_ids=input_ids,
                past_key_values=cache,
                position_ids=position_ids,
                use_cache=True,
            )
        
        logits = out.logits[0, -1, :] / temperature
        probs = torch.softmax(logits, dim=-1)
        
        if top_k > 0:
            indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
            logits[indices_to_remove] = float('-inf')
            probs = torch.softmax(logits, dim=-1)
        
        next_token = torch.multinomial(probs, num_samples=1)
        generated.append(next_token.item())
        
        if next_token.item() == tokenizer.eos_token_id:
            break
        
        input_ids = next_token.unsqueeze(0)
        cache = out.past_key_values
        current_position += 1
    
    return tokenizer.decode(generated, skip_special_tokens=True)


def main():
    # Setup
    device = "cpu"
    model = AutoModelForCausalLM.from_pretrained("google/gemma-3-270m").to(device)
    tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-270m")
    
    prompts = [
        "What is machine learning?",
        "Can you tell me a joke about cats.",
        "Do you know anything about India?"
    ]
    
    print("=" * 60)
    print("PROMPT PACKING WITH KV CACHE EXTRACTION")
    print("=" * 60)
    
    # Step 1: Pack prompts
    print("\nStep 1: Pack prompts into single sequence")
    packed_input_ids, prompt_ranges, prompt_lengths = pack_prompts(prompts, tokenizer, device)
    seq_len = packed_input_ids.shape[1]
    
    print(f"  Packed sequence length: {seq_len}")
    print(f"  Prompt ranges: {prompt_ranges}")
    for i, (prompt, length) in enumerate(zip(prompts, prompt_lengths)):
        print(f"    Prompt {i+1}: '{prompt}' -> {length} tokens")
    
    # Step 2: Create block-diagonal attention mask
    print("\nStep 2: Build block-diagonal attention mask")
    num_heads = model.config.num_attention_heads
    attn_mask = create_block_diagonal_mask(seq_len, prompt_ranges, num_heads, device)
    print(f"  Attention mask shape: {attn_mask.shape}")
    
    # Step 3: Single forward pass with packed prompts
    print("\nStep 3: Forward pass with prompt packing")
    with torch.no_grad():
        outputs = model(
            input_ids=packed_input_ids,
            attention_mask=attn_mask,
            use_cache=True,
        )
    print("  ✓ Forward pass complete")
    
    # Step 4: Extract individual KV caches
    print("\nStep 4: Extract KV caches for each prompt")
    all_caches = extract_prompt_caches(outputs.past_key_values, prompt_ranges)
    for i, (start, end) in enumerate(prompt_ranges):
        print(f"  Prompt {i+1}: Extracted cache with {end-start} tokens")
    
    # Step 5: Generate from each cache
    print("\n" + "=" * 60)
    print("Step 5: Generate from extracted caches")
    print("=" * 60)
    
    for i, (prompt, cache) in enumerate(zip(prompts, all_caches)):
        print(f"\nPrompt {i+1}: '{prompt}'")
        result = generate_from_cache(
            prompt_text=prompt,
            cache_list=cache,
            tokenizer=tokenizer,
            model=model,
            device=device,
            max_new=50,
            temperature=0.8,
            top_k=50
        )
        print(f"Generated: {result}\n")
    
    print("=" * 60)
    print("✓ Prompt packing with KV cache extraction complete!")
    print("=" * 60)


if __name__ == "__main__":
    main()

PROMPT PACKING WITH KV CACHE EXTRACTION

Step 1: Pack prompts into single sequence
  Packed sequence length: 26
  Prompt ranges: [(0, 6), (7, 17), (18, 26)]
    Prompt 1: 'What is machine learning?' -> 6 tokens
    Prompt 2: 'Can you tell me a joke about cats.' -> 10 tokens
    Prompt 3: 'Do you know anything about India?' -> 8 tokens

Step 2: Build block-diagonal attention mask
  Attention mask shape: torch.Size([1, 4, 26, 26])

Step 3: Forward pass with prompt packing
  ✓ Forward pass complete

Step 4: Extract KV caches for each prompt
  Prompt 1: Extracted cache with 6 tokens
  Prompt 2: Extracted cache with 10 tokens
  Prompt 3: Extracted cache with 8 tokens

Step 5: Generate from extracted caches

Prompt 1: 'What is machine learning?'
Generated: 
Machine learning is a technique that uses algorithms to extract or train information from data that is available in the real world. This technique is often used in various fields, including computer vision, natural language processing, an

In [130]:
import numpy as np
from typing import Dict, List, Tuple, Optional
import time

class SimpleAttention:
    """Simplified attention mechanism with KV caching"""
    
    def __init__(self, d_model=64, n_heads=4):
        self.d_model = d_model
        self.n_heads = n_heads
        self.d_k = d_model // n_heads
        
        # Initialize weight matrices
        np.random.seed(42)
        self.W_q = np.random.randn(d_model, d_model) * 0.01
        self.W_k = np.random.randn(d_model, d_model) * 0.01
        self.W_v = np.random.randn(d_model, d_model) * 0.01
        self.W_o = np.random.randn(d_model, d_model) * 0.01
        
    def compute_attention(self, Q, K, V):
        """Compute scaled dot-product attention"""
        scores = np.matmul(Q, K.T) / np.sqrt(self.d_k)
        attention_weights = self._softmax(scores)
        output = np.matmul(attention_weights, V)
        return output, attention_weights
    
    def _softmax(self, x):
        exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=-1, keepdims=True)
    
    def forward(self, x, kv_cache=None):
        """
        Forward pass with optional KV caching
        
        Args:
            x: Input embeddings [seq_len, d_model]
            kv_cache: Dict with 'keys' and 'values' or None
            
        Returns:
            output, new_kv_cache, computation_stats
        """
        seq_len = x.shape[0]
        
        # Compute Q, K, V
        Q = np.matmul(x, self.W_q)
        K_new = np.matmul(x, self.W_k)
        V_new = np.matmul(x, self.W_v)
        
        stats = {
            'tokens_processed': seq_len,
            'tokens_computed': seq_len,
            'tokens_from_cache': 0
        }
        
        # Handle KV cache
        if kv_cache is not None and 'keys' in kv_cache:
            # Concatenate cached K, V with new ones
            K = np.concatenate([kv_cache['keys'], K_new], axis=0)
            V = np.concatenate([kv_cache['values'], V_new], axis=0)
            
            stats['tokens_from_cache'] = kv_cache['keys'].shape[0]
            stats['tokens_computed'] = seq_len
            
            print(f"  ✓ Reusing {stats['tokens_from_cache']} tokens from cache")
            print(f"  ✓ Computing {stats['tokens_computed']} new tokens")
        else:
            K = K_new
            V = V_new
            print(f"  ✓ Computing all {seq_len} tokens (no cache)")
        
        # Compute attention
        output, attention_weights = self.compute_attention(Q, K, V)
        
        # Project output
        output = np.matmul(output, self.W_o)
        
        # Create new cache with ALL K, V (including new ones)
        new_cache = {
            'keys': K,
            'values': V
        }
        
        return output, new_cache, stats


class KVCacheDemo:
    """Demonstration of KV caching with two prompts"""
    
    def __init__(self, d_model=64):
        self.d_model = d_model
        self.attention = SimpleAttention(d_model=d_model)
        self.vocab = {}
        self.embeddings = {}
        
    def tokenize(self, text: str) -> List[str]:
        """Simple tokenization by splitting on spaces"""
        return text.split()
    
    def get_embedding(self, token: str) -> np.ndarray:
        """Get or create embedding for a token"""
        if token not in self.embeddings:
            # Create random embedding for new token
            self.embeddings[token] = np.random.randn(self.d_model) * 0.1
        return self.embeddings[token]
    
    def encode(self, tokens: List[str]) -> np.ndarray:
        """Convert tokens to embeddings"""
        return np.array([self.get_embedding(token) for token in tokens])
    
    def process_prompt(self, prompt: str, kv_cache: Optional[Dict] = None, 
                      prompt_name: str = "Prompt") -> Tuple[np.ndarray, Dict, Dict]:
        """Process a prompt with optional KV cache"""
        print(f"\n{'='*60}")
        print(f"{prompt_name}: '{prompt}'")
        print('='*60)
        
        # Tokenize
        tokens = self.tokenize(prompt)
        print(f"Tokens: {tokens}")
        print(f"Number of tokens: {len(tokens)}")
        
        # Encode
        x = self.encode(tokens)
        print(f"Embedding shape: {x.shape}")
        
        # Forward pass with timing
        start_time = time.time()
        output, new_cache, stats = self.attention.forward(x, kv_cache)
        elapsed = time.time() - start_time
        
        print(f"\nComputation Stats:")
        print(f"  - Total tokens: {stats['tokens_processed']}")
        print(f"  - Tokens from cache: {stats['tokens_from_cache']}")
        print(f"  - Tokens computed: {stats['tokens_computed']}")
        if stats['tokens_from_cache'] > 0:
            savings = (stats['tokens_from_cache'] / stats['tokens_processed']) * 100
            print(f"  - Cache savings: {savings:.1f}%")
        print(f"  - Time elapsed: {elapsed*1000:.2f}ms")
        print(f"\nOutput shape: {output.shape}")
        print(f"Cache contains: {new_cache['keys'].shape[0]} token K/V pairs")
        
        return output, new_cache, stats


def main():
    """Main demonstration"""
    print("=" * 60)
    print("KV CACHE DEMONSTRATION")
    print("=" * 60)
    print("\nThis demo shows how KV caching optimizes overlapping prompts")
    print("by reusing computed Key-Value pairs from previous tokens.\n")
    
    # Initialize demo
    demo = KVCacheDemo(d_model=64)
    
    # Define prompts
    prompt1 = "What is Machine Learning?"
    prompt2 = "What is Machine Learning and Deep Learning?"
    
    print(f"\nPrompt 1: {prompt1}")
    print(f"Prompt 2: {prompt2}")
    
    # Identify shared prefix
    tokens1 = demo.tokenize(prompt1)
    tokens2 = demo.tokenize(prompt2)
    shared_tokens = [t for t in tokens1 if t in tokens2]
    print(f"\nShared tokens: {shared_tokens}")
    print(f"Shared prefix length: {len(shared_tokens)} tokens")
    
    # Process Prompt 1 (no cache)
    output1, cache1, stats1 = demo.process_prompt(prompt1, kv_cache=None, 
                                                   prompt_name="PROMPT 1")
    
    # Process Prompt 2 WITHOUT cache (for comparison)
    print("\n" + "="*60)
    print("BASELINE: Processing Prompt 2 WITHOUT cache")
    print("="*60)
    output2_no_cache, _, stats2_no_cache = demo.process_prompt(
        prompt2, kv_cache=None, prompt_name="PROMPT 2 (No Cache)"
    )
    
    # Process Prompt 2 WITH cache from Prompt 1
    print("\n" + "="*60)
    print("OPTIMIZED: Processing Prompt 2 WITH cache from Prompt 1")
    print("="*60)
    
    # Extract only the NEW tokens for Prompt 2
    new_tokens = [t for t in tokens2 if t not in tokens1]
    new_prompt = " ".join(new_tokens)
    
    print(f"New tokens only: {new_tokens}")
    output2_cached, cache2, stats2_cached = demo.process_prompt(
        new_prompt, kv_cache=cache1, prompt_name="PROMPT 2 (With Cache)"
    )
    
    # Summary
    print("\n" + "="*60)
    print("PERFORMANCE COMPARISON")
    print("="*60)
    print(f"\nPrompt 2 without cache:")
    print(f"  - Tokens computed: {stats2_no_cache['tokens_computed']}")
    print(f"\nPrompt 2 with cache:")
    print(f"  - Tokens from cache: {stats2_cached['tokens_from_cache']}")
    print(f"  - Tokens computed: {stats2_cached['tokens_computed']}")
    total_tokens = stats2_cached['tokens_from_cache'] + stats2_cached['tokens_computed']
    savings = (stats2_cached['tokens_from_cache'] / total_tokens) * 100
    print(f"  - Total savings: {savings:.1f}%")
    
    print("\n" + "="*60)
    print("KEY INSIGHTS")
    print("="*60)
    print("""
1. Without cache: Must compute K,V for ALL tokens in Prompt 2
2. With cache: Reuse K,V for shared prefix, only compute new tokens
3. Memory cost: Store cached K,V pairs (increases linearly)
4. Compute savings: Proportional to shared prefix length
5. Use cases: Chat history, auto-completion, beam search
    """)
    
    print("\n" + "="*60)
    print("CACHE STRUCTURE")
    print("="*60)
    print(f"\nCache after Prompt 1:")
    print(f"  - Keys shape: {cache1['keys'].shape}")
    print(f"  - Values shape: {cache1['values'].shape}")
    print(f"  - Represents {cache1['keys'].shape[0]} cached tokens")
    
    print(f"\nCache after Prompt 2:")
    print(f"  - Keys shape: {cache2['keys'].shape}")
    print(f"  - Values shape: {cache2['values'].shape}")
    print(f"  - Represents {cache2['keys'].shape[0]} cached tokens")
    print(f"  - Growth: +{cache2['keys'].shape[0] - cache1['keys'].shape[0]} tokens")


if __name__ == "__main__":
    main()

KV CACHE DEMONSTRATION

This demo shows how KV caching optimizes overlapping prompts
by reusing computed Key-Value pairs from previous tokens.


Prompt 1: What is Machine Learning?
Prompt 2: What is Machine Learning and Deep Learning?

Shared tokens: ['What', 'is', 'Machine', 'Learning?']
Shared prefix length: 4 tokens

PROMPT 1: 'What is Machine Learning?'
Tokens: ['What', 'is', 'Machine', 'Learning?']
Number of tokens: 4
Embedding shape: (4, 64)
  ✓ Computing all 4 tokens (no cache)

Computation Stats:
  - Total tokens: 4
  - Tokens from cache: 0
  - Tokens computed: 4
  - Time elapsed: 12.49ms

Output shape: (4, 64)
Cache contains: 4 token K/V pairs

BASELINE: Processing Prompt 2 WITHOUT cache

PROMPT 2 (No Cache): 'What is Machine Learning and Deep Learning?'
Tokens: ['What', 'is', 'Machine', 'Learning', 'and', 'Deep', 'Learning?']
Number of tokens: 7
Embedding shape: (7, 64)
  ✓ Computing all 7 tokens (no cache)

Computation Stats:
  - Total tokens: 7
  - Tokens from cache: 0
  - 